# Main 

Zexuan: The main file for category ML model production

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sqlite3
from sqlalchemy.engine import create_engine

from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from tpot import TPOTClassifier

import tpot_functions 
import periods 
import utils


Define some key variables and numbers

In [18]:
target = 'deal_probability'
LIMIT = None
TIMEOUT_MINS = 30
#SCORING = 'neg_mean_squared_error'
RS = 27

In [26]:
train = pd.read_csv('data/train.csv')[:LIMIT]
train_pd = pd.read_csv('train_price_normalized.csv')[:LIMIT]
#train_periods = pd.read_csv('data/periods_aggregate_features_items.csv')

In [27]:
city = preprocessing.LabelEncoder()
city.fit(train['city'])  
list(city.classes_)
train['city'] = city.transform(train['city'])

In [28]:
train = pd.concat([train_pd[['item_id', 'price', 'norm_price', target]],
                  pd.get_dummies(train[['category_name', 'user_type', 'region', 'city']])], 
                  axis=1) 

#train = pd.concat(pd.get_dummies(train[['category_name', 'user_type', target]]), train[['item_id', 'price', target]]
      
    
    

In [29]:
train

,item_id,price,norm_price,deal_probability,city,category_name_Автомобили,category_name_Аквариум,category_name_Аудио и видео,category_name_Билеты и путешествия,category_name_Бытовая техника,...,region_Саратовская область,region_Свердловская область,region_Ставропольский край,region_Татарстан,region_Тульская область,region_Тюменская область,region_Удмуртия,region_Ханты-Мансийский АО,region_Челябинская область,region_Ярославская область
0,b912c3c6a6ad,400.0,-0.001144,0.12789,460,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2dac0150717d,3000.0,-0.006929,0.00000,1300,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ba83aefab5dc,4000.0,-0.002041,0.43177,1276,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,02996f1dd2ea,2200.0,-0.001126,0.80323,940,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,7c90be56d2ab,40000.0,-0.002323,0.20797,317,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,51e0962387f7,1300.0,-0.001135,0.80323,1657,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,c4f260a2b48a,11000.0,-0.001434,0.00000,975,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,6b71309d6a8a,500.0,-0.002104,0.80323,1153,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,c5b969cb63a2,500.0,-0.002104,0.00000,1107,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,b1570962e68c,400.0,-0.004198,0.00000,975,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
#train_new = pd.merge(train, train_periods, on='item_id')
#train_new

In [32]:
train = train.dropna()
train

,item_id,price,norm_price,deal_probability,city,category_name_Автомобили,category_name_Аквариум,category_name_Аудио и видео,category_name_Билеты и путешествия,category_name_Бытовая техника,...,region_Саратовская область,region_Свердловская область,region_Ставропольский край,region_Татарстан,region_Тульская область,region_Тюменская область,region_Удмуртия,region_Ханты-Мансийский АО,region_Челябинская область,region_Ярославская область
0,b912c3c6a6ad,400.0,-0.001144,0.12789,460,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2dac0150717d,3000.0,-0.006929,0.00000,1300,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ba83aefab5dc,4000.0,-0.002041,0.43177,1276,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,02996f1dd2ea,2200.0,-0.001126,0.80323,940,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,7c90be56d2ab,40000.0,-0.002323,0.20797,317,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,51e0962387f7,1300.0,-0.001135,0.80323,1657,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,c4f260a2b48a,11000.0,-0.001434,0.00000,975,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,6b71309d6a8a,500.0,-0.002104,0.80323,1153,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,c5b969cb63a2,500.0,-0.002104,0.00000,1107,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,b1570962e68c,400.0,-0.004198,0.00000,975,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
X = (train.drop(['item_id',target], axis=1)).values
y = train[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.75, test_size=0.25)
#X_train = X
#y_train = y

In [17]:
# TimeSeriesSplit NOT RIGHT YET
#tss = TimeSeriesSplit(n_splits=4)
#train_index, test_index = list(tss.split(X))[-1]
#X_train, X_test = X[train_index], X[test_index]
#y_train, y_test = y[train_index], y[test_index]

TPOT functions

In [18]:
#tpot_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
#                      random_state=42, verbosity=2)

In [21]:
#tpot_optimizer = tpot_functions.TpotAutoml(mode='regression', max_time_mins=TIMEOUT_MINS, 
#                                           scoring=SCORING, random_state=RS, n_jobs=-1,verbosity=2,
#                                           generations = 2, population_size=15)


In [20]:
#tpot_optimizer.fit(X_train, y_train)

In [ ]:
#top_scores = tpot_optimizer.get_top_models(return_scores=True)
#print('\ntop cv scores:')
#print(top_scores)
#print('\ntop models')
#print(tpot_optimizer.top_models)
#print('\nthe best test score:')
#test_score = tpot_optimizer.score(X_test, y_test)
#print(test_score)

Try GradientBoostingRegressor model 

In [38]:
#GBR = GradientBoostingRegressor(alpha=0.85, learning_rate=0.1, loss="ls",
#                                              max_features=0.9, min_samples_leaf=5,
#                                              min_samples_split=6)

In [41]:
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
config_dict = {'sklearn.ensemble.GradientBoostingRegressor': {
        'n_estimators': [100, 200, 400],
        'loss': ["ls", "lad", "huber", "quantile"],
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'subsample': np.arange(0.05, 1.01, 0.05),
        'max_features': np.arange(0.05, 1.01, 0.05),
        'alpha': [0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
    }}

GBR  = GradientBoostingRegressor(**params)
RSCV = RandomizedSearchCV(estimator=GBR, random_state=RS,
                          param_distributions=config_dict['sklearn.ensemble.GradientBoostingRegressor'],
                         n_iter=5,
                         scoring='r2',
                         verbose=2)




In [42]:
RSCV.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] subsample=0.2, n_estimators=400, min_samples_split=2, min_samples_leaf=11, max_features=0.9500000000000001, max_depth=3, loss=ls, learning_rate=1.0, alpha=0.75 
[CV]  subsample=0.2, n_estimators=400, min_samples_split=2, min_samples_leaf=11, max_features=0.9500000000000001, max_depth=3, loss=ls, learning_rate=1.0, alpha=0.75, total= 8.4min
[CV] subsample=0.2, n_estimators=400, min_samples_split=2, min_samples_leaf=11, max_features=0.9500000000000001, max_depth=3, loss=ls, learning_rate=1.0, alpha=0.75 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.5min remaining:    0.0s


[CV]  subsample=0.2, n_estimators=400, min_samples_split=2, min_samples_leaf=11, max_features=0.9500000000000001, max_depth=3, loss=ls, learning_rate=1.0, alpha=0.75, total= 8.4min
[CV] subsample=0.2, n_estimators=400, min_samples_split=2, min_samples_leaf=11, max_features=0.9500000000000001, max_depth=3, loss=ls, learning_rate=1.0, alpha=0.75 
[CV]  subsample=0.2, n_estimators=400, min_samples_split=2, min_samples_leaf=11, max_features=0.9500000000000001, max_depth=3, loss=ls, learning_rate=1.0, alpha=0.75, total= 8.9min
[CV] subsample=0.9500000000000001, n_estimators=200, min_samples_split=18, min_samples_leaf=5, max_features=0.6500000000000001, max_depth=6, loss=huber, learning_rate=0.5, alpha=0.75 
[CV]  subsample=0.9500000000000001, n_estimators=200, min_samples_split=18, min_samples_leaf=5, max_features=0.6500000000000001, max_depth=6, loss=huber, learning_rate=0.5, alpha=0.75, total=17.9min
[CV] subsample=0.9500000000000001, n_estimators=200, min_samples_split=18, min_samples_le

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 124.4min finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'n_estimators': [100, 200, 400], 'loss': ['ls', 'lad', 'huber', 'quantile'], 'learning_rate': [0.001, 0.01, 0.1, 0.5, 1.0], 'max_depth': range(1, 11), 'min_samples_split': range(2, 21), 'min_samples_leaf': range(1, 21), 'subsample': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.3...0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]), 'alpha': [0.75, 0.8, 0.85, 0.9, 0.95, 0.99]},
  

In [44]:
mse = mean_squared_error(y_test, RSCV.predict(X_test))
print("MSE: %.4f" % mse)

MSE: 0.0567


In [45]:
# Does not make sense
r2 = r2_score(y_test, RSCV.predict(X_test))
print("R2: %.4f" % r2)

R2: 0.1490


In [46]:
results = RSCV.predict(X_test)
results

array([0.06331569, 0.07438474, 0.26803051, ..., 0.05375978, 0.25107914,
       0.10053434])

In [47]:
# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(RSCV.staged_predict(X_test)):
    test_score[i] = RSCV.loss_(y_test, y_pred)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, RSCV.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

AttributeError: 'RandomizedSearchCV' object has no attribute 'staged_predict'

Load test.csv file and compute 

In [64]:
test_normalized = pd.read_csv('test_price_normalized.csv')[:LIMIT]
test_all = pd.read_csv('data/test.csv')[:LIMIT]

In [67]:
test_all.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,6544e41a8817,dbe73ad6e4b5,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Отдам бесплатно,На ангарском,NaN,66,2017-04-18,Private,a8b57acb5ab304f9c331ac7a074219aed4d349d8aef386...,2020.0
1,65b9484d670f,2e11806abe57,Свердловская область,Нижняя Тура,Хобби и отдых,Велосипеды,Дорожные,NaN,NaN,Продам велосипед,"Продам велосипед KAMA F200,в нормальном состо...",3000.0,4,2017-04-16,Private,NaN,NaN
2,8bab230b2ecd,0b850bbebb10,Новосибирская область,Бердск,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,NaN,NaN,BBK,Продам новый телевизор BBK 32 диагональ смарт...,15000.0,15,2017-04-17,Private,8c361112cb049745ef2d1b0ae73594fc5c107286b0c942...,2960.0
3,8e348601fefc,5f1d5c3ce0da,Саратовская область,Саратов,Для дома и дачи,Бытовая техника,Для кухни,Вытяжки,NaN,Вытяжка Jetair 60,"Продам новую вытяжку в упаковке,с документами....",4500.0,70,2017-04-17,Private,NaN,NaN
4,8bd2fe400b89,23e2d97bfc7f,Оренбургская область,Бузулук,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Коляска зима-лето,Продам отличную коляску. б/у 1 год. все вопрос...,4900.0,15,2017-04-15,Private,bc3cf6deef10840fc302e38eb48fa7748aa1e28d534f8f...,1002.0


In [69]:
city = preprocessing.LabelEncoder()
city.fit(test_all['city'])  
list(city.classes_)
test_all['city'] = city.transform(test_all['city'])

In [70]:
test2 = pd.concat([test_normalized[['item_id', 'price', 'norm_price']], 
                  pd.get_dummies(test_all[['category_name', 'user_type', 'region', 'city']])], 
                  axis=1) 
test2.head()

,item_id,price,norm_price,city,category_name_Автомобили,category_name_Аквариум,category_name_Аудио и видео,category_name_Билеты и путешествия,category_name_Бытовая техника,category_name_Велосипеды,...,region_Саратовская область,region_Свердловская область,region_Ставропольский край,region_Татарстан,region_Тульская область,region_Тюменская область,region_Удмуртия,region_Ханты-Мансийский АО,region_Челябинская область,region_Ярославская область
0,6544e41a8817,NaN,NaN,306,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,65b9484d670f,3000.0,-0.024347,933,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,8bab230b2ecd,15000.0,-0.001982,147,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8e348601fefc,4500.0,-0.055562,1238,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,8bd2fe400b89,4900.0,-0.001098,236,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
test2 = test2.dropna()
test_id = test2['item_id']
test2 = test2.drop(['item_id'], axis=1)

test2_probability = RSCV.predict(test2)
test2_probability


array([0.44002462, 0.19378906, 0.40419167, ..., 0.16690717, 0.03135473,
       0.14093651])

In [72]:
print(len(test2))
print(len(test2_probability))
print(len(test_id))

477853
477853
477853


In [73]:
d = {'item_id': test_id, 'deal_probability': test2_probability}
test_output = pd.DataFrame(data=d)
test_output

,item_id,deal_probability
1,65b9484d670f,0.440025
2,8bab230b2ecd,0.193789
3,8e348601fefc,0.404192
4,8bd2fe400b89,0.257298
5,c63dbd6c657f,0.233169
6,6d1a410df86e,0.106489
7,e8d3e7922b80,0.009335
8,2bc1ab208462,0.063526
9,7e05d77a9181,0.055259
10,e2ec19a3ca2b,0.411414


In [74]:
test_output.to_csv('tabular.csv')